In [7]:
# %% [markdown]
# # 셀 1: 환경 세팅 (transformers 설치)
# - 오프라인/내부망에서도 모델이 캐시에 있으면 동작합니다.
# - 처음 실행 시 모델 다운로드가 필요할 수 있습니다.

import sys
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import json, re, time, math, random
from pathlib import Path
from typing import List, Tuple, Optional
from googletrans import Translator

print("Python:", sys.version)
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


Python: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
Torch: 2.5.1+cu121
CUDA available: True


In [8]:
# %% [markdown]
# # 셀 2: 경로 설정 및 JSON 로드

# ✅ 입력 JSON 경로 (사용자 요청 경로)
INPUT_PATH = Path(r"C:\POLO\polo-system\models\math\_build_yolo\equations_explained.json")

# ✅ 출력 JSON 경로 (원본 옆에 *.ko.json)
OUTPUT_PATH = INPUT_PATH.with_name(INPUT_PATH.stem + ".ko.json")

with open(INPUT_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

assert "overview" in data and "items" in data, "JSON에 overview/items 키가 없습니다."
print("로드 완료! items 개수:", len(data.get("items", [])))


로드 완료! items 개수: 19


In [17]:
# %% [markdown]
# # 셀 3 (수정): 수식 전용 보호/복원 (PUA 토큰)

import re
from typing import List, Tuple

# 수식 환경 패턴 (비탐욕, DOTALL)
_MATH_ENV_NAMES = r"(?:equation|align|gather|multline|eqnarray|cases)"
MATH_PATTERN = re.compile(
    r"(?P<DOLLAR2>\${2}[\s\S]*?\${2})"               # $$ ... $$
    r"|(?P<DOLLAR1>(?<!\\)\$[\s\S]*?(?<!\\)\$)"      # $ ... $  (이스케이프된 \$ 제외)
    r"|(?P<BRACK>\\\[[\s\S]*?\\\])"                  # \[ ... \]
    r"|(?P<PAREN>\\\([\s\S]*?\\\))"                  # \( ... \)  ← 여기 수정
    r"|(?P<BEGIN>\\begin\{"+_MATH_ENV_NAMES+r"\}\*?[\s\S]*?\\end\{"+_MATH_ENV_NAMES+r"\}\*?)",
    re.DOTALL
)

# 유니코드 Private-Use 영역 토큰 (번역기가 건드리지 못하게)
_PUA_START = "\uE000"
_PUA_END   = "\uE001"
def _make_token(i: int) -> str:
    return f"{_PUA_START}{i:06d}{_PUA_END}"

def protect_math(text: str) -> Tuple[str, List[str]]:
    """LaTeX 수식만 PUA 토큰으로 치환하여 보호"""
    protected: List[str] = []
    out = []
    last = 0
    for m in MATH_PATTERN.finditer(text):
        out.append(text[last:m.start()])
        seg = m.group(0)
        idx = len(protected)
        protected.append(seg)
        out.append(_make_token(idx))
        last = m.end()
    out.append(text[last:])
    return "".join(out), protected

def restore_math(text: str, protected: List[str]) -> str:
    """PUA 토큰을 원본 수식으로 복원"""
    def repl(m):
        idx = int(m.group(1))
        return protected[idx] if 0 <= idx < len(protected) else m.group(0)
    return re.sub(fr"{_PUA_START}(\d{{6}}){_PUA_END}", repl, text)

def has_hangul(s: str) -> bool:
    return bool(re.search(r"[가-힣]", s or ""))


In [18]:
# %% [markdown]
# # 셀 4 (교체): googletrans 번역기 + 재시도/청크 + 수식 원형 보장
# - 번역 전: 수식 보호(PUA 토큰)
# - 번역 후: PUA 토큰 → 원문 수식 완전 복원
# - raise_Exception 버그/429 등을 재시도로 회피

import time, random
from typing import List, Optional
from googletrans import Translator

translator = Translator(service_urls=[
    "translate.googleapis.com",
    "translate.google.com",
    "translate.google.co.kr",
])

def _translate_paragraph(p: str) -> str:
    # googletrans 대용량 문단 방지: 너무 길면 문장 단위로 분할
    if len(p) > 4000:
        sentences = re.split(r"(?<=[\.\?\!])\s+", p)
        out = []
        for s in sentences:
            if s.strip():
                out.append(translator.translate(s, src="en", dest="ko").text)
        return " ".join(out)
    else:
        return translator.translate(p, src="en", dest="ko").text

def translate_en2ko_google(text: str, max_retry: int = 5, sleep_base: float = 0.8) -> str:
    # 이미 한글 포함 시(혼용 텍스트)라도: "수식은 그대로" 유지가 더 중요하므로
    # 1) 수식 보호 → 2) (비수식)영문만 번역 → 3) 복원
    if not text:
        return text

    # 1) 수식 보호
    prot_text, math_buf = protect_math(text)

    # 2) 단락 단위 번역 (빈 줄 유지)
    parts = re.split(r"(\n\s*\n)", prot_text)  # 구분자 유지
    out_parts: List[str] = []

    for p in parts:
        # 공백/빈단락은 그대로
        if not p.strip() or re.fullmatch(r"\n\s*\n", p):
            out_parts.append(p)
            continue

        # PUA 토큰만으로 이루어진 조각은 그대로 둠 (수식 블록 자체)
        if re.fullmatch(fr"(?:{_PUA_START}\d{{6}}{_PUA_END})+", p):
            out_parts.append(p)
            continue

        # 재시도 번역
        last_err: Optional[Exception] = None
        for attempt in range(1, max_retry + 1):
            try:
                # 수식 토큰은 그대로 두고 나머지 텍스트만 번역
                # (단락 내부에 수식 토큰이 섞여 있을 수 있음)
                chunks = re.split(fr"({_PUA_START}\d{{6}}{_PUA_END})", p)
                tr_chunks = []
                for ch in chunks:
                    if re.fullmatch(fr"{_PUA_START}\d{{6}}{_PUA_END}", ch):
                        tr_chunks.append(ch)  # 수식 토큰은 그대로
                    else:
                        # 이미 한글이 섞여 있으면 그대로 두는 편이 안전
                        tr_chunks.append(_translate_paragraph(ch) if not has_hangul(ch) and ch.strip() else ch)
                out_parts.append("".join(tr_chunks))
                break
            except Exception as e:
                last_err = e
                delay = sleep_base * (2 ** (attempt - 1)) + random.uniform(0, 0.25)
                time.sleep(delay)
        else:
            print(f"⚠️ 번역 실패 (원문 유지): {repr(last_err)}")
            out_parts.append(p)

    merged = "".join(out_parts)

    # 3) 수식 복원
    restored = restore_math(merged, math_buf)

    return restored


In [19]:
# %% [markdown]
# # 셀 5: overview / items[*].explanation 번역 실행
import copy

out = copy.deepcopy(data)

# 1) overview -> overview_ko
out["overview_ko"] = translate_en2ko_google(data.get("overview", "") or "")

# 2) 각 item의 explanation -> explanation_ko
for it in out.get("items", []):
    exp = it.get("explanation", "") or ""
    it["explanation_ko"] = translate_en2ko_google(exp)

print("번역 완료!")


⚠️ 번역 실패 (원문 유지): AttributeError("'Translator' object has no attribute 'raise_Exception'")


KeyboardInterrupt: 

In [14]:
# %% [markdown]
# # 셀 6: 저장 및 미리보기

with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(out, f, ensure_ascii=False, indent=2)

print("저장 경로:", OUTPUT_PATH)

# 간단 미리보기
print("\n=== overview_ko (앞 400자) ===")
print((out.get("overview_ko") or "")[:400])

if out.get("items"):
    print("\n=== items[0].explanation_ko (앞 400자) ===")
    print((out["items"][0].get("explanation_ko") or "")[:400])


저장 경로: C:\POLO\polo-system\models\math\_build_yolo\equations_explained.ko.json

=== overview_ko (앞 400자) ===
어시스턴트
당신은 일반적인 기술 청중을 남기는 명확하고 간결한 기술 저술입니다.

=== items[0].explanation_ko (앞 400자) ===
어시스턴트
주어진 방정식을 단계별로 분해합시다.

### 예
방정식은 다음과 같습니다.
[[prot_0]]

### 설명
이 방정식을 명확하게 설명하기 위해, 우리는 그것을 구성 요소와 그 역할로 분류 할 수 있습니다.

1. ** 기호 ** :이 기호는 곱셈을 나타냅니다.그것은 우리가 그 자체로 숫자 448을 곱하고 있음을 알려줍니다.
2. ** 기호 ** :이 기호는 숫자 448을 나타냅니다. 우리가 곱하는 숫자입니다.

따라서, [[prot_2]] 방정식은 우리가 그 자체로 숫자 448을 곱하고 있음을 의미합니다.다시 말해, 우리는 448의 제곱을 찾고 있습니다.

### 결론
이 방정식의 핵심 목적은 숫자 448의 제곱을 계산하는 것입니다. 논문의 맥락에서, 이것은 448 개의 단위 인 정사각형의 영
